<a href="https://colab.research.google.com/github/eduswiss/deep-learning-with-tensorflow/blob/master/notebooks/11_regression_in_tensorflow_with_tensorboard.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Regression Example in TensorFlow with TensorBoard

Predicting house prices in Boston, Massachusetts while logging results for TensorBoard

## Load dependencies

Add import for `TensorBoard`

In [1]:
import numpy as np
from tensorflow.keras.datasets import boston_housing 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout 
from tensorflow.keras.layers import BatchNormalization 
from tensorflow.keras.callbacks import ModelCheckpoint 
from tensorflow.keras.callbacks import TensorBoard # new!  
import os

## Load data

In [2]:
(X_train, y_train), (X_valid, y_valid) = boston_housing.load_data()

In [3]:
X_train.shape

(404, 13)

In [4]:
X_valid.shape

(102, 13)

In [5]:
X_train[0]

array([  1.23247,   0.     ,   8.14   ,   0.     ,   0.538  ,   6.142  ,
        91.7    ,   3.9769 ,   4.     , 307.     ,  21.     , 396.9    ,
        18.72   ])

In [6]:
y_train[0]

15.2

## Design neural network architecture

In [7]:
model = Sequential()

model.add(Dense(32, input_dim=13, activation='relu'))
model.add(BatchNormalization())

model.add(Dense(16, activation='relu'))
model.add(BatchNormalization())
# model.add(Dropout(0.2))
model.add(Dropout(0.5))

model.add(Dense(1, activation='linear'))

In [8]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 32)                448       
_________________________________________________________________
batch_normalization (BatchNo (None, 32)                128       
_________________________________________________________________
dense_1 (Dense)              (None, 16)                528       
_________________________________________________________________
batch_normalization_1 (Batch (None, 16)                64        
_________________________________________________________________
dropout (Dropout)            (None, 16)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 17        
Total params: 1,185
Trainable params: 1,089
Non-trainable params: 96
_____________________________________________________

## Configure Model

In [9]:
model.compile(loss='mean_squared_error', optimizer='nadam')

In [10]:
output_dir = 'model_output/'

In [11]:
run_name = 'regression_baseline'
output_path = output_dir + run_name

In [12]:
if not os.path.exists(output_path):
    os.makedirs(output_path)

In [13]:
modelcheckpoint = ModelCheckpoint(output_path + '/weights.{epoch:02d}.hdf5', # decimal integers
                                  save_weights_only=True)

Create TensorBoard callback.

In [14]:
tensorboard = TensorBoard(log_dir='logs/' + run_name)

## Train!

Add `TensorBoard` callback to the `fit()` function.

In [15]:
model.fit(X_train, y_train, 
          batch_size=8, epochs=32, verbose=1, 
          validation_data=(X_valid, y_valid),
          callbacks=[modelcheckpoint, tensorboard])

Epoch 1/32
 1/51 [..............................] - ETA: 0s - loss: 630.6904WARNING:tensorflow:From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/summary_ops_v2.py:1277: stop (from tensorflow.python.eager.profiler) is deprecated and will be removed after 2020-07-01.
Instructions for updating:
use `tf.profiler.experimental.stop` instead.
51/51 [==============================] - 0s 6ms/step - loss: 578.5701 - val_loss: 527.7381
Epoch 2/32
51/51 [==============================] - 0s 2ms/step - loss: 566.7703 - val_loss: 577.8136
Epoch 3/32
51/51 [==============================] - 0s 2ms/step - loss: 554.7806 - val_loss: 594.1018
Epoch 4/32
51/51 [==============================] - 0s 2ms/step - loss: 537.9033 - val_loss: 571.4468
Epoch 5/32
51/51 [==============================] - 0s 2ms/step - loss: 522.5831 - val_loss: 551.6610
Epoch 6/32
51/51 [==============================] - 0s 2ms/step - loss: 501.9771 - val_loss: 531.2451
Epoch 7/32
51/51 [===========================

## Performing inference

In [16]:
model.load_weights(output_path + '/weights.28.hdf5')

In [17]:
X_valid[42]

array([  9.32909,   0.     ,  18.1    ,   0.     ,   0.713  ,   6.185  ,
        98.7    ,   2.2616 ,  24.     , 666.     ,  20.2    , 396.9    ,
        18.13   ])

In [18]:
y_valid[42]

14.1

In [19]:
model.predict(np.reshape(X_valid[42], [1, 13]))

array([[12.582474]], dtype=float32)